In [13]:
def fitness_function(chromosome, distance_matrix, 
                     battery_capacity=100, consumption_rate=1, charging_rate=1,
                     w1=0.6, w2=0.4):
    """
    Menghitung nilai fitness:
    - Kendaraan hanya bisa isi daya di SPKLU (Sx)
    - Saat di SPKLU, kendaraan mengisi seminimal mungkin agar bisa sampai ke SPKLU atau depot berikutnya
    - Fitness = w1 * D + w2 * C
    
    Parameter:
    -----------
    chromosome : list
        Contoh: ['D1','C1','S1','C2','C3','S2','D1']
    distance_matrix : dict
        Dictionary jarak antar node, contoh: distance_matrix[('C1','C2')] = 12.3
    battery_capacity : float
        Kapasitas maksimum baterai kendaraan
    consumption_rate : float
        Konsumsi energi per km
    charging_rate : float
        Kecepatan charging (energi per menit)
    w1, w2 : float
        Bobot komponen fitness
    
    Return:
    --------
    fitness : float
        Nilai fitness total (semakin kecil semakin baik)
    """

    total_distance = 0
    total_charging_time = 0
    invalid_penalty = -1  # nilai besar kalau baterai tidak cukup dan tidak bisa ngecas

    # Pisahkan rute berdasarkan delimiter '|'
    routes = []
    current_route = []
    for gene in chromosome:
        if gene == '|':
            if current_route:
                routes.append(current_route)
                current_route = []
        else:
            current_route.append(gene)
    if current_route:
        routes.append(current_route)

    # Simulasi per rute
    for route in routes:
        battery = battery_capacity

        for i in range(len(route) - 1):
            a, b = route[i], route[i + 1]

            # Ambil jarak
            if (a, b) in distance_matrix:
                dist = distance_matrix[(a, b)]
            elif (b, a) in distance_matrix:
                dist = distance_matrix[(b, a)]
            else:
                continue  # skip jika jarak tidak diketahui

            total_distance += dist
            needed_energy = dist * consumption_rate

            
            if 'S' in a:
                # Di SPKLU, isi secukupnya agar bisa sampai ke SPKLU/depot berikutnya
                # Cari indeks SPKLU/depot berikutnya di depan
                print(f"At charging station {a} with battery {battery}")
                energy_needed_to_next_station = 0
                for j in range(i, len(route) - 1):
                    u, v = route[j], route[j + 1]
                    if (u, v) in distance_matrix:
                        energy_needed_to_next_station += distance_matrix[(u, v)] * consumption_rate
                    elif (v, u) in distance_matrix:
                        energy_needed_to_next_station += distance_matrix[(v, u)] * consumption_rate
                    if 'S' in v or 'D' in v:  # berhenti kalau ketemu SPKLU/depot
                        break

                # Hitung tambahan energi yang dibutuhkan
                required_energy = max(0, energy_needed_to_next_station - battery)
                charge_time = required_energy / charging_rate
                total_charging_time += charge_time
                battery += required_energy
                battery = min(battery, battery_capacity)
                print(f"Charging at {a}: +{required_energy} energy, Time: {charge_time} min, Battery: {battery}")

            # Cek apakah cukup baterai untuk melanjutkan
            if battery < needed_energy:
                print(f"Insufficient battery from {a} to {b}. Battery: {battery}, Needed: {needed_energy}")
                # Kalau tidak cukup dan bukan di SPKLU → solusi invalid
               
                return invalid_penalty
            # Setelah bergerak ke titik berikutnya, kurangi energi
            battery -= needed_energy

    # Normalisasi sederhana (opsional)
    print("Total Distance:", total_distance, "Total Charging Time:", total_charging_time)
    D = total_distance / 100.0
    C = total_charging_time / 100.0

    # Hitung fitness
    fitness = w1 * D + w2 * C
    return fitness


# ==== Contoh data ====
chromosome = ['D1', 'C1', 'S1', 'C2', 'C3', 'S2', 'D1']
chromosome2= ['D1', 'C1', 'C2', 'S1', 'C3', 'S2', 'D1']

distance_matrix = {
    ('D1','C1'): 40, ('C1','S1'): 30, ('S1','C2'): 50, ('C2','C3'): 30,
    ('C3','S2'): 20, ('S2','D1'): 40
}

# Uji fungsi
fitness_value = fitness_function(
    chromosome,
    distance_matrix,
    battery_capacity=100,  # maksimal 100 energi
    consumption_rate=1,    # 1 energi/km
    charging_rate=2        # 2 energi/menit
)
print("Fitness value:", fitness_value)


At charging station S1 with battery 30
Charging at S1: +70 energy, Time: 35.0 min, Battery: 100
At charging station S2 with battery 0
Charging at S2: +40 energy, Time: 20.0 min, Battery: 40
Total Distance: 210 Total Charging Time: 55.0
Fitness value: 1.48
